<a href="https://colab.research.google.com/github/codypape-dev/ml-notebooks/blob/main/PML_Paula_Perdomo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Creacion de modelo predictivo para determinar la demanda sobre el uso de un sistema de alquiler de bicicletas**



---

> ## Paula Perdomo
> #### Asignatura: Principios de Machine Learning
> #### Universidad de los Andes - Maestria en Inteligencia Artificial
> ![Universidad de los Andes](https://uniandes.edu.co/sites/default/files/logo-header.png)

---


Este proyecto pretende crear un modelo predictivo que permita determinar la demanda sobre el uso de un sistema de alquiler de bicicletas. Este conocimiento puede dar soporte para mejorar el servicio y conocer los factores que inciden en su eficiencia. Fomentar planes de movilidad sostenible es una manera de reducir las emisiones de CO2, que afectan la temperatura del planeta y desequilibran el ciclo natural.

El conjunto de datos recoge información sobre la cantidad de bicicletas rentadas en un período de tiempo, junto con información meteorológica y de temporalidad, entre otros

### Objetivos

- Aplicar técnicas de regresión para construir un modelo predictivo que permita estimar la demanda sobre el uso de un sistema de alquiler de bicicletas siguiendo el ciclo de machine learning.

- Determinar cuáles son los factores que más inciden en la demanda con base en los datos.

## 1. Exploración y perfilamiento de los datos



### Importación de librerías requeridas

Importaremos la librería Pandas, Seaborn  y, además, utilizaremos la librería scikit-learn.

In [ ]:
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

### Carga de datos
Realizaremos la carga de datos usando la función de Pandas read_csv(). En este caso agregaremos un nuevo parámetro para especificar el separador del archivo que, para nuestro conjunto de datos, es una coma:

In [ ]:
data_raw = pd.read_csv('/content/Datos_Etapa-1.csv', sep=',')

Veremos el tamaño de los datos usando shape:

In [ ]:
data_raw.shape

(17379, 9)

Y los primeros datos del conjunto usando `head()`:

In [ ]:
data_raw.head()

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0,1,Night


La informacion contenida en la base de datos se puede interpretar segun este diccionario de datos:


---
| **Columna** | **Tipo**   | **Descripción**                                 |
|-------------|------------|-------------------------------------------------|
| season      | categórica | Estación del año (Winter, Spring, Summer, Fall) |
| weekday     | numérico   | Día de la semana (de 1 a 7)                     |
| weathersit  | categórica | Clima (Clear, Mist, Light Rain, Heavy Rain)     |
| temp        | numérico   | Temperatura                                     |
| atemp       | numérico   | Sensación de temperatura                        |
| hum         | numérico   | Humedad                                         |
| windspeed   | numérico   | Velocidad del viento                            |
| cnt         | numérico   | Cantidad de bicicletas rentadas                 |
| time_of_day | categórica | Parte del día (Morning, Evening, Night)         |


## 2. Limpieza y preparación de los datos

Antes de entrenar cualquier modelo de aprendizaje automático, es importante asegurar que los datos utilizados no contienen errores, como datos faltantes o duplicados. Para esto es recomendable usar una nueva variable, en este caso `data`, que usaremos para almacenar un conjunto de datos modificado:



In [ ]:
data = data_raw.copy()

### Eliminación de datos faltantes

A traves de las funciones `isna()` verificamos si hay celdas vacias:

In [ ]:
data.isna()

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
17374,False,False,False,False,False,False,False,False,False
17375,False,False,False,False,False,False,False,False,False
17376,False,False,False,False,False,False,False,False,False
17377,False,False,False,False,False,False,False,False,False


Si adicionamos la función `sum()`, obtendremos la cantidad de datos faltantes por variable:

In [ ]:
data.isna().sum()

season         0
weekday        0
weathersit     0
temp           0
atemp          0
hum            0
windspeed      0
cnt            0
time_of_day    0
dtype: int64

Asi podemos verificar que no hay datos vacios

### Eliminacion de duplicados

Usar la función duplicated() para verificar si hay filas duplicadas en la base de datos:

In [ ]:
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
17374    False
17375    False
17376    False
17377    False
17378    False
Length: 17379, dtype: bool

Agregamos `sum()` para obtener el número de filas duplicadas:

In [ ]:
data.duplicated().sum()

42

Ya que encontramos que existen filas duplicadas, las eliminamos con el metodo `drop_duplicates()`

In [ ]:
data = data.drop_duplicates()

### Eliminacion de variables poco relevantes

 Para preparar los datos para este paso, se realizara un la transformación de las variables categóricas `season`, `weathersit` y `time_of_day`

In [ ]:
data = pd.get_dummies(data, columns= ['weekday', 'season', 'weathersit', 'time_of_day'])

data.head()

,temp,atemp,hum,windspeed,cnt,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,...,season_Spring,season_Summer,season_Winter,weathersit_Clear,weathersit_Heavy Rain,weathersit_Light Rain,weathersit_Mist,time_of_day_Evening,time_of_day_Morning,time_of_day_Night
0,3.28,3.0014,0.81,0.0,16,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
1,2.34,1.9982,0.80,0.0,40,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
2,2.34,1.9982,0.80,0.0,32,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
3,3.28,3.0014,0.75,0.0,13,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
4,3.28,3.0014,0.75,0.0,1,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1


In [ ]:
data.describe()

,temp,atemp,hum,windspeed,cnt,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,...,season_Spring,season_Summer,season_Winter,weathersit_Clear,weathersit_Heavy Rain,weathersit_Light Rain,weathersit_Mist,time_of_day_Evening,time_of_day_Morning,time_of_day_Night
count,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,...,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000,17337.000000
mean,15.373723,15.419489,0.626924,12.744552,189.865836,0.143912,0.142643,0.140855,0.142355,0.142239,...,0.253792,0.258984,0.243756,0.656919,0.000173,0.081329,0.261579,0.294053,0.334372,0.371575
std,9.046797,11.338895,0.192857,8.196736,181.400275,0.351010,0.349718,0.347882,0.349423,0.349305,...,0.435193,0.438089,0.429360,0.474752,0.013154,0.273348,0.439507,0.455629,0.471784,0.483240
min,-7.060000,-16.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.980000,5.997800,0.480000,7.001500,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15.500000,15.996800,0.630000,12.998000,143.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,23.020000,24.999200,0.780000,16.997900,281.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,39.000000,50.000000,1.000000,56.996900,977.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### División de datos

Vamos a separar la variable objetivo y las variables independientes para, posteriormente, crear los conjuntos de entrenamiento y pruebas:

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=77)

# 3. Construcción del modelo de regresión polinomial

Definiremos los conjuntos de entrenamiento y se escalan con RobustScaler

In [ ]:
x_train = train.drop(['cnt'],axis=1)
y_train = train['cnt']
columns = x_train.columns
scaler = RobustScaler()
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=columns)

Primero vamos a definir una variable con el nombre `polynomial_regression`, que se utilizara para realizar la búsqueda de hiperparámetros:

In [ ]:
polynomial_regression = make_pipeline(
    PolynomialFeatures(),
    LinearRegression()
)

Utilizaremos un objeto de la clase `KFold`, con el que definiremos 10 subconjuntos sobre el conjunto de entrenamiento:

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)

El siguiente paso es definir el espacio de búsqueda del hiperparámetro.

In [ ]:
valores = [2, 3]
param_grid = {'polynomialfeatures__degree': valores}

Finalmente, el último paso antes de realizar la búsqueda de hiperparámetros es crear el objeto de tipo GridSearchCV.

In [ ]:
grid = GridSearchCV(polynomial_regression, param_grid, scoring='neg_root_mean_squared_error', cv=kfold, n_jobs=-1)

A continuación, realiza la búsqueda de hiperparámetros utilizando el conjunto de entrenamiento, compuesto por las variables x_train y y_train.



In [ ]:
grid.fit(x_train, y_train)
print("Mejor parámetro: ", grid.best_params_)

mejor_modelo = grid.best_estimator_

Mejor parámetro:  {'polynomialfeatures__degree': 3}


In [ ]:
x_test = test.drop(['cnt'],axis=1)
y_test = test['cnt']
x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test, columns=columns)

Realiza predicciones con el fin de compararlas con los valores reales almacenados en `y_test`.


In [ ]:
y_pred = mejor_modelo.predict(x_test)

Evaluacion del modelo

In [ ]:
rmse_pr = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE: ", rmse_pr)

RMSE:  130.59121868780232


In [ ]:
mae_pr = mean_absolute_error(y_test, y_pred)
print("MAE: ", mae_pr)

MAE:  94.63842632641291


In [ ]:
r2_pr = r2_score(y_test, y_pred)
print('R²: ', r2_pr)

R²:  0.4926797460135195


# 4. Construcción del modelo de regresión regularizada Lasso

Definiremos los conjuntos de entrenamiento y se escalan con StandardScaler



In [ ]:
x_train = train.drop(['cnt'],axis=1)
y_train = train['cnt']
columns = x_train.columns
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=columns)

Utilizaremos un objeto de la clase `KFold`, con el que definiremos 10 subconjuntos sobre el conjunto de entrenamiento:

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state = 0)

El siguiente paso es definir el espacio de búsqueda del hiperparámetro.

In [ ]:
valores = [1, 2, 3, 4, 5]
param_grid = {'alpha': valores}

Finalmente, el último paso antes de realizar la búsqueda de hiperparámetros es crear el objeto de tipo GridSearchCV.

In [ ]:
lasso = Lasso(max_iter=2000)
grid = GridSearchCV(lasso, param_grid, scoring='neg_root_mean_squared_error', cv=kfold, n_jobs=-1, refit=True)

A continuación, realiza la búsqueda de hiperparámetros utilizando el conjunto de entrenamiento, compuesto por las variables x_train y y_train.

In [ ]:
grid.fit(x_train, y_train)
print("Mejor parámetro: ", grid.best_params_)

mejor_modelo = grid.best_estimator_

Mejor parámetro:  {'alpha': 1}


In [ ]:
list(zip(x_train.columns, grid.best_estimator_.coef_))

[('temp', 39.19198103839863),
 ('atemp', 21.842442433534824),
 ('hum', -27.646109700166683),
 ('windspeed', -0.0),
 ('weekday_0', -2.426884847183268),
 ('weekday_1', -0.5682387396568973),
 ('weekday_2', -0.0),
 ('weekday_3', 0.0),
 ('weekday_4', 0.0),
 ('weekday_5', 0.5062339724906546),
 ('weekday_6', 0.6723201777679971),
 ('season_Fall', 22.548732507426372),
 ('season_Spring', 8.413040807874985),
 ('season_Summer', -2.3028523693715317),
 ('season_Winter', -2.3308152040123638),
 ('weathersit_Clear', 1.1937844567950469),
 ('weathersit_Heavy Rain', 0.0),
 ('weathersit_Light Rain', -10.596877373349308),
 ('weathersit_Mist', -0.0),
 ('time_of_day_Evening', 39.645655203382525),
 ('time_of_day_Morning', 0.0),
 ('time_of_day_Night', -48.6539913410163)]

In [ ]:
x_test = test.drop(['cnt'],axis=1)
y_test = test['cnt']
x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test, columns=columns)

Realiza predicciones con el fin de compararlas con los valores reales almacenados en `y_test`.

In [ ]:
y_pred_lr = mejor_modelo.predict(x_test)

Evaluacion del modelo

In [ ]:
rmse_lr = mean_squared_error(y_test, y_pred_lr, squared=False)
print("RMSE: ", rmse_lr)

RMSE:  139.73457240621644


In [ ]:
mae_lr = mean_absolute_error(y_test, y_pred_lr)
print("MAE: ", mae_lr)

MAE:  103.60265964565383


In [ ]:
r2_lr = r2_score(y_test, y_pred_lr)
print('R²: ', r2_lr)

R²:  0.41915267678849677


Construccion de modelo para seleccion de variables considerando alpha=5, el numero mas grande de la lista de valores establecidos para este taller

In [ ]:
lasso_sv = Lasso(alpha=5)
lasso_sv.fit(x_train, y_train)
list(zip(x_train.columns, lasso_sv.coef_))

[('temp', 24.59974638862015),
 ('atemp', 26.813435119766734),
 ('hum', -26.32779252106779),
 ('windspeed', 0.0),
 ('weekday_0', -0.0),
 ('weekday_1', -0.0),
 ('weekday_2', -0.0),
 ('weekday_3', 0.0),
 ('weekday_4', 0.0),
 ('weekday_5', 0.0),
 ('weekday_6', 0.0),
 ('season_Fall', 12.238926532557342),
 ('season_Spring', 0.819234343469567),
 ('season_Summer', -0.0),
 ('season_Winter', -8.897543541009995),
 ('weathersit_Clear', 0.0),
 ('weathersit_Heavy Rain', -0.0),
 ('weathersit_Light Rain', -7.47172606297155),
 ('weathersit_Mist', -0.0),
 ('time_of_day_Evening', 39.27664685085888),
 ('time_of_day_Morning', 0.0),
 ('time_of_day_Night', -46.10338277889293)]

#5. Tabla comparativa del rendimiento de los modelos de regresion

In [ ]:
from prettytable import PrettyTable
t = PrettyTable(['Modelo', 'R²', 'MAE', 'RMSE'])
t.add_row(['Polinomial', '{0:.2f}'.format(r2_pr), '{0:.2f}'.format(mae_pr), '{0:.2f}'.format(rmse_pr)])
t.add_row(['Lasso', '{0:.2f}'.format(r2_lr), '{0:.2f}'.format(mae_lr), '{0:.2f}'.format(rmse_lr)])
print(t)

+------------+------+--------+--------+
|   Modelo   |  R²  |  MAE   |  RMSE  |
+------------+------+--------+--------+
| Polinomial | 0.49 | 94.64  | 130.59 |
|   Lasso    | 0.42 | 103.60 | 139.73 |
+------------+------+--------+--------+




# Análisis de resultados



---


>
1. ¿Cuál es el grado de la transformación polinomial que fue seleccionado utilizando la técnica de validación?

> Mejor parámetro para regresión polinomial:  `{'polynomialfeatures__degree': 3}`


2. ¿Cuál fue el valor de α que fue seleccionado utilizando la técnica de validación para la regresión Lasso?

> Mejor parámetro para regresión Lasso `{'alpha': 1}`

3. A partir de la tabla comparativa, ¿cuál modelo ofrece el mejor rendimiento sobre el conjunto test? ¿Qué interpretación puedes darles a los valores obtenidos sobre las métricas de rendimiento?

> El mejor rendimiento lo obtuvo la regresion polinomial que se ajusta al 49% de los datos, sin embargo sigue siendo un numero muy bajo y no se recomienda el uso de este modelo para hacer predicciones. Ademas considerando que la variable `cnt` tiene el 50% de sus valores por debajo de 149, el error del modelo es muy grande con MAE 94.64  y RMSE 130.59.

4. ¿Cuáles variables fueron seleccionadas con el modelo Lasso?

| Variable   | Coeficiente |
|------------|:-----------:|
| temp | 24.60 |
| atemp | 26.81 |
| hum | -26.33 |
| windspeed | 0.0 |
| weekday_0 | -0.0 |
| weekday_1 | -0.0 |
| weekday_2 | -0.0 |
| weekday_3 | 0.0 |
| weekday_4 | 0.0 |
| weekday_5 | 0.0 |
| weekday_6 | 0.0 |
| season_Fall | 12.24 |
| season_Spring | 0.82 |
| season_Summer | -0.0 |
| season_Winter | -8.90 |
| weathersit_Clear | 0.0 |
| weathersit_Heavy Rain | -0.0 |
| weathersit_Light Rain | -7.47 |
| weathersit_Mist | -0.0 |
| time_of_day_Evening | 39.28 |
| time_of_day_Morning | 0.0 |
| time_of_day_Night | -46.10 |


> A partir de estas, ¿qué interpretación de cara al problema puedes dar?

De estos resultados se puede interpretar que las variables con coeficiente igual a 0 son poco relevantes para el modelo de regresión Lasso.

Estas son: windspeed, weekday_0, weekday_1, weekday_2, weekday_3, weekday_4, weekday_5, weekday_6, season_Summer, weathersit_Clear, weathersit_Heavy Rain, weathersit_Mist, time_of_day_Morning.

Según los resultados de esta selección de variables se puede inferir que la variable `windspeed` es de poca relevancia para el modelo y no se tiene en cuenta para realizar predicciones.

Sin embargo **la mayoría de las variables son categorías que salen del proceso de transformación de variables categóricas** y pues nos permiten interpretar las categorías que tienen mucha o poca relevancia en la predicción. Por ejemplo al evaluar la variable categórica `time_of_day` podemos interpretar que las personas no le dan mucha importancia a considerar si el momento para usar la bicicleta es en la mañana; sin embargo aumenta el uso de bicicletas en la tarde. Por el contrario en la noche disminuye.






